In [1]:
import os
import regex as re
import pandas as pd

In [2]:
from collections import Counter

In [3]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger_ru')

[nltk_data] Downloading package punkt to /home/varsey/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/varsey/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger_ru to
[nltk_data]     /home/varsey/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_ru is already up-to-
[nltk_data]       date!


True

In [4]:
from rapidfuzz import process as proc
from rapidfuzz import fuzz

In [5]:
from tqdm import tqdm

In [6]:
import re

In [9]:
import yake

In [10]:
flt_chars = [
    ')', '(', ',', '!', '', ':', '|', '>', '<', '[', ']', '...', './', '/\n', '\xa0',
    '«', '»', '_', '?', '~', '=', '\\', ';', '\n', '``', "''", '✅', '/n', '/', '..', '`', "'", '-',
    '✔', '⇒', '"', '$', '%', '&', "'", ',', '^', '`', '{', '}', "('",
]

In [11]:
stop_words = (nltk.corpus.stopwords.words('russian')
              + nltk.corpus.stopwords.words('english')
              + nltk.corpus.stopwords.words('azerbaijani'))
exclude = [x for x in set(stop_words + flt_chars) if len(x) > 0] + ['pet']
exclude.remove('up')
exclude.remove('s')
exclude.remove('ya')
exclude = set(exclude)

In [12]:
'ya' in exclude

False

In [13]:
remove_chars = {
    '-': ' ',
    "'": ' ',
    "(": ' ',
    ")": ' ',
    '"': ' ',
    '<': ' ',
    '>': ' ',
    "/": ' ',
}
replace_chars = {
    '*': ' x ',
    "%": 'pct',
    ",": '.',
    " s ":  's '
}


In [14]:
def simple_process_item(x: str):
    x = x.lower()
    for char in remove_chars.keys():
        x = x.replace(char, ' ')
    for k, v in replace_chars.items():
        x = x.replace(k, v)
    item = ' '.join(re.split(r'(\d+)', x))
    while item.count(2 * " ") > 0:
        item = item.replace(2 * " ", " ")
    item = item.replace(' .', '.').replace('. ', '.')
    return ' '.join([x for x in item.split() if x not in exclude])

In [15]:
def count_common_words(str1, str2):
    set1 = set([x for x in str1.lower().split() if len(x) > 3 and not x.isdigit()])
    set2 = set([x for x in str2.lower().split() if len(x) > 3 and not x.isdigit()])
    if ((len(set1) + len(set2))/2) == 0:
        return 0
    return len(set1.intersection(set2))/((len(set1) + len(set2))/2)

In [16]:
def count_common_digits(str1, str2):
    set1 = set([x for x in str1.lower().split() if x.isdigit()])
    set2 = set([x for x in str2.lower().split() if x.isdigit()])
    if ((len(set1) + len(set2))/2) == 0:
        return 0
    return len(set1.intersection(set2))/((len(set1) + len(set2))/2)

In [17]:
def count_digit_share(x: str):
    counter = 0
    for char in x:
        if char.isdigit():
            counter += 1
    return counter/len(x.replace(' ', '')) if len(x) > len(x.replace(' ', '')) and counter > 0 else 0.5

In [18]:
def remove_numbers(text):
    text = text.lower()
    text = re.sub(r'\d+', '', text)
    text = ' '.join(text.split('.'))
    return ' '.join([x for x in text.split() if len(x) > 2 and x not in ('pct', 'dad')][:3])

In [20]:
data_folder = '/'.join(os.getcwd().split('/')[:-1])
data_folder

'/home/varsey/PycharmProjects/drl-case'

In [21]:
# Loading data
df_incoming = pd.read_csv(f'{data_folder}/data/incoming.csv')
df_outcoming = pd.read_csv(f'{data_folder}/data/outcoming.csv')

In [22]:
df_outcoming['unit'] = df_incoming['Unit'].str.lower()
df_incoming['unit'] = df_incoming['Unit'].str.lower()

In [23]:
text = df_outcoming.sample(1)['Product or Service Name'].values[0]
print(text)
simple_process_item(text)

QLAVPRODUKT BADIMCAN KURUSU 450 QR 1X12


'qlavprodukt badimcan kurusu 450 qr 1 x 12'

In [24]:
df_outcoming['items'] = df_outcoming['Product or Service Name'].apply(lambda x: simple_process_item(x))
df_incoming['items'] = df_incoming['Product or Service Name'].apply(lambda x: simple_process_item(x))

In [25]:
Counter(" ".join(df_outcoming['items']).split()).most_common(50)

[('1', 5396),
 ('x', 5072),
 ('gr', 3748),
 ('12', 1452),
 ('pct', 1055),
 ('sevimli', 771),
 ('dad', 771),
 ('24', 637),
 ('pinar', 590),
 ('lays', 488),
 ('200', 480),
 ('6', 424),
 ('20', 397),
 ('180', 382),
 ('lt', 332),
 ('president', 324),
 ('500', 290),
 ('150', 286),
 ('qr', 267),
 ('90', 266),
 ('10', 257),
 ('15', 256),
 ('30', 256),
 ('suxari', 250),
 ('kesmik', 249),
 ('18', 247),
 ('ml', 234),
 ('xama', 228),
 ('25', 218),
 ('ulker', 199),
 ('8', 192),
 ('40', 191),
 ('16', 174),
 ('tadim', 172),
 ('50', 171),
 ('kg', 169),
 ('250', 165),
 ('labne', 157),
 ('bebeto', 156),
 ('0', 154),
 ('pendir', 152),
 ('60', 147),
 ('peynir', 145),
 ('300', 144),
 ('100', 142),
 ('kefir', 142),
 ('130', 138),
 ('4', 137),
 ('yagi', 136),
 ('9', 133)]

In [26]:
%%time

to_find = df_incoming['items'].to_list()[:1000]
candidates = set(df_outcoming['items'].to_list())

outcoming, incoming1, incoming2, score1, score2, score_w, score_d = [], [], [], [], [], [], []
for item in tqdm(to_find[:]):
    res1 = proc.extract(
                item,
                candidates,
                scorer=fuzz.partial_token_sort_ratio,
                limit=1
    )[0]
    res2 = proc.extract(
                item,
                candidates,
                scorer=fuzz.partial_ratio,
                limit=1
    )[0]
    outcoming.append(item)
    incoming1.append(res1[0])
    incoming2.append(res2[0])
    score1.append(res1[1])
    score2.append(res2[1])
    if len(res1) > 0:
        score_w.append(count_common_words(item, res1[0]))
        score_d.append(count_common_digits(item, res1[0]) / count_digit_share(res1[0]))


100%|██████████| 1000/1000 [00:02<00:00, 494.18it/s]

CPU times: user 2.02 s, sys: 6.23 ms, total: 2.03 s
Wall time: 2.03 s


In [27]:
comp = pd.DataFrame()
comp['right'] = outcoming
comp['left1'] = incoming1
comp['left2'] = incoming2
comp['score1'] = score1
comp['score2'] = score2
comp['score3'] = 2*comp['score1'] *comp['score2'] /(comp['score1'] + comp['score2'] )
comp['score_w'] = score_w
comp['score_d'] = score_d
comp['score_d'] = comp['score_d'] /10

In [28]:
# parq_write(
#     f'{os.getcwd()}/comp.parq',
#     comp,
#     compression='GZIP'
# )

In [29]:
# comp_parq = ParquetFile(f'{os.getcwd()}/comp.parq',)
# comp = comp_parq.to_pandas()

In [30]:
df_outcoming.shape, df_incoming.shape, comp.shape

((5000, 7), (5000, 7), (1000, 8))

In [31]:
sub_result = pd.concat(
    [
        df_incoming[:1000],
        comp
    ],
    axis=1,
)
sub_result.shape

(1000, 15)

In [32]:
compilation_full = sub_result.merge(df_outcoming, how='left', left_on='left1', right_on='items')
compilation_full = compilation_full.drop_duplicates(subset=['#_x']) 
compilation_full.shape

(1000, 22)

In [33]:
compilation_full = compilation_full[[
          'items_x', 'items_y',
          'Quantity/Volume_x', 'unit_x', 'Quantity/Volume_y', 'unit_y',
          'score1', 'score2', 'score3', 'score_w', 'score_d',
          'Product or Service Name_x', 'Product or Service Name_y', ]]

In [34]:
comp_true = compilation_full[
    (compilation_full.score3 > 78) 
    # & (compilation_full.score_w > 0.0)
    & (compilation_full.score_d > 0.17)
]

In [35]:
comp_doubt = compilation_full.drop(comp_true.index, axis=0)

In [36]:
comp_doubt['items_y']  = ''
comp_doubt['Quantity/Volume_y'] = 0
comp_doubt['unit_y'] = ''
# comp_doubt['Product or Service Name_y'] = ''

In [37]:
comp_true.shape[0] / comp_doubt.shape[0]

20.27659574468085

In [38]:
comp_doubt.sample(5)

,items_x,items_y,Quantity/Volume_x,unit_x,Quantity/Volume_y,unit_y,score1,score2,score3,score_w,score_d,Product or Service Name_x,Product or Service Name_y
989,lays cili laym 150 gr,,108,ədəd,0,,72.222222,72.222222,72.222222,0.333333,0.000000,LAYS CILI LAYM 150GR,LAYS PAPRIKA CIPS 50 GR
3855,prezident xama 15 pct 200 gr,,9,ədəd,0,,90.566038,92.857143,91.697281,0.500000,0.000000,PREZIDENT XAMA 15% 200 GR,PRESIDENT XAMA 20% 350 GR
6988,aqua minerale qazsiz 2 lt 1 x 6,,18,adet.lt,0,,62.745098,74.285714,68.029436,0.000000,0.560000,AQUA MINERALE QAZSIZ 2 LT 1X6,MILLA AYRAN NANELI 1L 1X6
2395,hazer.lok.small cut rose lemn menth flv.350 gr...,,16,eded,0,,67.415730,66.666667,67.039106,0.250000,0.136667,HAZER.LOK.SMALL CUT ROSE LEMN MENTH FLV.350GR ...,HAZER.LOK.PISTACHIO ROSE LEMON 454 GR 1X12 (4132)
2875,kardelen fritur yagi kizartmalik 18 lt 1 x 1,,5,adet.lt,0,,61.111111,78.571429,68.750000,0.000000,0.213333,KARDELEN FRITUR YAGI(KIZARTMALIK) 18LT PET 1X1,LAYS SIR 150 GR 1 x 18


In [39]:
compilation_scored = pd.concat([comp_true, comp_doubt])

In [40]:
compilation_scored['unit_common'] = compilation_scored['unit_x'] + ' ' +  compilation_scored['unit_y']

In [41]:
compilation_scored['unit_common_set'] = compilation_scored['unit_common'].apply(lambda x: set(str(x).replace('.', ' ').split()))

In [42]:
compilation_scored['unit_common_str'] = compilation_scored['unit_common_set'].apply(
    lambda x: str(list(x)[0]) if len(list(x)) == 0 else ' '.join(list(x))
)

In [43]:
compilation_scored['quant_in'] = compilation_scored['Quantity/Volume_x'].astype('str').str.replace(',', '.').fillna(0).astype('float')
compilation_scored['quant_out'] = compilation_scored['Quantity/Volume_y'].astype('str').str.replace(',', '.').fillna(0).astype('float')

In [44]:
# x - incoming y - outcoming
compilation_scored[[
          'items_x', 'items_y',
          'score3', 'score1', 'score2', 'score_w', 'score_d', 'Product or Service Name_y']].sample(250)

,items_x,items_y,score3,score1,score2,score_w,score_d,Product or Service Name_y
4040,pinar ketcap acili plastik 600 gr 1 x 6,pinar ketcap acili plastik 600 gr,100.0,100.0,100.0,1.0,0.466667,PINAR KETCAP ACILI PLASTIK 600 GR
854,bebeto super belts 75 gr 1 x 12,bebeto super belts 75 gr 1 x 12,100.0,100.0,100.0,1.0,0.480000,BEBETO SUPER BELTS 75 GR 1X12
4606,president sirin k dolce erik 20 pct 200 gr 1 x 12,president sirin k dolce erik 20 pct 200 gr 1 x 12,100.0,100.0,100.0,1.0,0.475000,PRESIDENT SIRIN K/M DOLCE ERIK 20% 200GR 1X12
1166,sevimli dad kesmik gaysili 9 pct 180 gr 1 x 1,sevimli dad kesmik gaysili 9 pct 180 gr 1 x 1,100.0,100.0,100.0,1.0,0.583333,SEVIMLI DAD KESMIK GAYSILI 9% 180 GR 1x1
2617,ulker pendir labne 400 gr 1 x 6,ulker pendir labne 400 gr 1 x 6,100.0,100.0,100.0,1.0,0.480000,ULKER PENDIR LABNE 400 GR 1X6
...,...,...,...,...,...,...,...,...
8415,qlavprodukt gobelek opyata 580 qr 1 x 12,qlavprodukt gobelek opyata 580 qr 1 x 12,100.0,100.0,100.0,1.0,0.550000,QLAVPRODUKT GOBELEK OPYATA 580 QR 1X12
3256,doktor milk kesmik 0 pct 180 qr uzumlu 1 x 1,doktor milk kesmik 0 pct 180 qr uzumlu 1 x 1,100.0,100.0,100.0,1.0,0.566667,DOKTOR MILK KESMIK 0% 180 QR UZUMLU 1x1
914,ulker puding icim ciyelekli 60 gr 1 x 24,ulker puding icim ciyelekli 60 gr 1 x 24,100.0,100.0,100.0,1.0,0.640000,ULKER PUDING ICIM CIYELEKLI 60 GR 1X24
3891,kirlangic riviera zeytinyagi 0.5 lt 1 x 18,kirlangic riviera zeytinyagi 0.5 lt 1 x 18,100.0,100.0,100.0,1.0,0.700000,"KIRLANGIC RIVIERA ZEYTINYAGI 0,5 LT PET 1X18"


In [45]:
compilation_scored['items_x_numberless'] = compilation_scored['items_x'].apply(lambda x: remove_numbers(x))

In [46]:
kw_extractor1 = yake.KeywordExtractor(
    n=3,
    dedupLim=0.5,
    dedupFunc='sequencematcher',
    windowsSize=2,
    top=3,
    features=None
)

kw_extractor2 = yake.KeywordExtractor(
    n=1,
    dedupLim=0.5,
    dedupFunc='sequencematcher',
    windowsSize=2,
    top=1,
    features=None
)

kw_extractor = yake.KeywordExtractor(
    n=2,
    dedupLim=0.5,
    dedupFunc='sequencematcher',
    windowsSize=2,
    top=3,
    features=None
)


In [47]:
def get_2main_word(word: str):
    keywords = kw_extractor.extract_keywords(word)
    x = {kw[0]: kw[1] for kw in keywords}
    if len(x) > 0:
        return min(x, key=x.get)
    else:
        return ''

def get_1main_word(words: str):
    if len(words) > 0:
        fltr = kw_extractor.extract_keywords(words)
        fltr_dct = {kw[0]: kw[1] for kw in fltr}
        return max(fltr_dct, key=fltr_dct.get)
    else:
        return ''

In [48]:
for item in compilation_scored.sample(50)['items_x_numberless']:
    words = get_2main_word(item)
    word = get_1main_word(words)
    print(item)
    print('---')
    splitted_words = words.split()
    if len(splitted_words) == 2:
        print(' '.join(splitted_words))
        print(' '.join(splitted_words[:-1]))
        # print(' '.join(splitted_words[-1:]))
    elif len(splitted_words) == 3:
        print(' '.join(splitted_words[:-1]))
        print(splitted_words[-1])
    # print(word)
    print()

tadim antep fistigi
---
tadim antep
tadim

ulker puding cokokrem
---
ulker puding
ulker

pinar salam bitir
---
pinar salam
pinar

halk kapkek meyveli
---
halk kapkek
halk

semeyniy budj mal
---
semeyniy budj
semeyniy

kardelen fritur yagi
---
kardelen fritur
kardelen

doktor milk teze
---
doktor milk
doktor

korolevskiy pinqvin limonad
---
korolevskiy pinqvin
korolevskiy

doritos taco
---
doritos taco
doritos

sevimli gatig suse
---
sevimli gatig
sevimli

pinar hindi sosis
---
pinar hindi
pinar

semeyniy budj mal
---
semeyniy budj
semeyniy

tadim badem ici
---
tadim badem
tadim

komili misir yagi
---
komili misir
komili

qlavprodukt pashtet toyuq
---
qlavprodukt pashtet
qlavprodukt

dolcia puding bananli
---
dolcia puding
dolcia

sevimli gatig
---
sevimli gatig
sevimli

bimba yoqurt albali
---
bimba yoqurt
bimba

sevimli kesmik yagli
---
sevimli kesmik
sevimli

ulker pendir labne
---
ulker pendir
ulker

frico gouda loaf
---
frico gouda
frico

semeyniy budj mal
---
semeyniy budj
semeyni

In [49]:
compilation_scored['main_2word'] = compilation_scored['items_x_numberless'].apply(lambda x: get_2main_word(x))
compilation_scored['main_1word'] = compilation_scored['main_2word'].apply(lambda x: get_1main_word(x))

In [50]:
compilation_scored.groupby(by=['main_1word', 'main_2word', 'items_x', 'items_y', 'unit_common_str']).agg({'quant_in':'sum','quant_out':'sum'}) # .sort_values(['unit_common_str'])

quant_in  \
main_1word main_2word         items_x                                            items_y                                            unit_common_str             
aci        berrak aci         berrak aci sus biberi tursusu 370 ml 1 x 12        berrak aci sus biberi tursusu 370 ml 1 x 12        adet eded ml         12.0   
acili      doritos acili      doritos acili 130 gr 1 x 16                        doritos acili 130 gr 1 x 16                        eded                 90.0   
                              doritos acili 74 gr 1 x 24                         doritos acili 74 gr 1 x 24                         eded                130.0   
adjika     qlavprodukt adjika qlavprodukt adjika aromatnaya 170 qr 1 x 15        qlavprodukt adjika aromatnaya 170 qr 1 x 15        eded                825.0   
                              qlavprodukt adjika domasnaya 170 gr                qlavprodukt adjika aromatnaya 170 gr               ədəd eded            10.0   
...                                                                                                                                                       ...   
zolotoy    zolotoy petusok    zolotoy petusok naggets pendir 300 gr 1 x 12 4608  zolotoy petusok naggets pendir 300 gr 1 x 12 4608  eded                 12.0   
                              zolotoy petusok naggets toyuqlu 300 gr 1 x 12 4639 zolotoy petusok naggets toyuqlu 300 gr 1 x 12 4639 eded                  6.0   
                              zolotoy petusok naggets vkus.300 gr 1 x 12 4719    zolotoy petusok naggets vkus.300 gr 1 x 12 4719    eded                 24.0   
                              zolotoy petusok strips vkusnyaski 300 gr 1 x 12... zolotoy petusok strips vkusnyaski 300 gr 1 x 12... eded                  3.0   
                              zolotoy petusok suxari toyuq steyk 280 gr 1 x 1... zolotoy petusok suxari toyuq steyk 280 gr 1 x 1... eded                 12.0   

                                                                                                                                                     quant_out  
main_1word main_2word         items_x                                            items_y                                            unit_common_str             
aci        berrak aci         berrak aci sus biberi tursusu 370 ml 1 x 12        berrak aci sus biberi tursusu 370 ml 1 x 12        adet eded ml           4.0  
acili      doritos acili      doritos acili 130 gr 1 x 16                        doritos acili 130 gr 1 x 16                        eded                  80.0  
                              doritos acili 74 gr 1 x 24                         doritos acili 74 gr 1 x 24                         eded                  12.0  
adjika     qlavprodukt adjika qlavprodukt adjika aromatnaya 170 qr 1 x 15        qlavprodukt adjika aromatnaya 170 qr 1 x 15        eded                  10.0  
                              qlavprodukt adjika domasnaya 170 gr                qlavprodukt adjika aromatnaya 170 gr               ədəd eded             90.0  
...                                                                                                                                                        ...  
zolotoy    zolotoy petusok    zolotoy petusok naggets pendir 300 gr 1 x 12 4608  zolotoy petusok naggets pendir 300 gr 1 x 12 4608  eded                   8.0  
                              zolotoy petusok naggets toyuqlu 300 gr 1 x 12 4639 zolotoy petusok naggets toyuqlu 300 gr 1 x 12 4639 eded                   4.0  
                              zolotoy petusok naggets vkus.300 gr 1 x 12 4719    zolotoy petusok naggets vkus.300 gr 1 x 12 4719    eded                   3.0  
                              zolotoy petusok strips vkusnyaski 300 gr 1 x 12... zolotoy petusok strips vkusnyaski 300 gr 1 x 12... eded                  12.0  
                              zolotoy petusok suxari toyuq steyk 280 gr 1 x 1... zolotoy petusok suxari toyuq steyk 280 gr 

In [51]:
compilation_scored.groupby(['main_1word']).agg({'quant_in': 'sum', 'quant_out': 'sum'})

,quant_in,quant_out
main_1word,,
aci,12.0,4.0
acili,220.0,92.0
adjika,840.0,106.0
alma,73.0,171.0
angry,44.0,375.0
...,...,...
xardal,320.0,623.0
xlorami,200.0,0.0
xren,42.0,50.0


In [52]:
brand_agg = compilation_scored.groupby(['main_1word', 'main_2word', 'items_x', 'items_y', 'unit_common_str']).agg({'quant_in': 'sum', 'quant_out': 'sum'}).reset_index()
brand_agg.rename(columns={'quant_in': 'brand_inventory_in', 'quant_out': 'brand_inventory_out'}, inplace=True)

# Aggregate at the item_type level within each brand group
type_agg = compilation_scored.groupby(['main_1word', 'main_2word']).agg({'quant_in': 'sum', 'quant_out': 'sum'}).reset_index()

# Merge the two aggregation results
result = pd.merge(brand_agg, type_agg, on='main_1word', how='left')

In [53]:
result

,main_1word,main_2word_x,items_x,items_y,unit_common_str,brand_inventory_in,brand_inventory_out,main_2word_y,quant_in,quant_out
0,aci,berrak aci,berrak aci sus biberi tursusu 370 ml 1 x 12,berrak aci sus biberi tursusu 370 ml 1 x 12,adet eded ml,12.0,4.0,berrak aci,12.0,4.0
1,acili,doritos acili,doritos acili 130 gr 1 x 16,doritos acili 130 gr 1 x 16,eded,90.0,80.0,doritos acili,220.0,92.0
2,acili,doritos acili,doritos acili 74 gr 1 x 24,doritos acili 74 gr 1 x 24,eded,130.0,12.0,doritos acili,220.0,92.0
3,adjika,qlavprodukt adjika,qlavprodukt adjika aromatnaya 170 qr 1 x 15,qlavprodukt adjika aromatnaya 170 qr 1 x 15,eded,825.0,10.0,qlavprodukt adjika,840.0,106.0
4,adjika,qlavprodukt adjika,qlavprodukt adjika domasnaya 170 gr,qlavprodukt adjika aromatnaya 170 gr,ədəd eded,10.0,90.0,qlavprodukt adjika,840.0,106.0
...,...,...,...,...,...,...,...,...,...,...
2762,zolotoy,zolotoy petusok,zolotoy petusok naggets pendir 300 gr 1 x 12 4608,zolotoy petusok naggets pendir 300 gr 1 x 12 4608,eded,12.0,8.0,zolotoy petusok,71.0,35.0
2763,zolotoy,zolotoy petusok,zolotoy petusok naggets toyuqlu 300 gr 1 x 12 ...,zolotoy petusok naggets toyuqlu 300 gr 1 x 12 ...,eded,6.0,4.0,zolotoy petusok,71.0,35.0
2764,zolotoy,zolotoy petusok,zolotoy petusok naggets vkus.300 gr 1 x 12 4719,zolotoy petusok naggets vkus.300 gr 1 x 12 4719,eded,24.0,3.0,zolotoy petusok,71.0,35.0
2765,zolotoy,zolotoy petusok,zolotoy petusok strips vkusnyaski 300 gr 1 x 1...,zolotoy petusok strips vkusnyaski 300 gr 1 x 1...,eded,3.0,12.0,zolotoy petusok,71.0,35.0


In [54]:
result.groupby(by=['main_1word', 'main_2word_x', 'items_x', 'items_y', 'unit_common_str']).agg(
    {'quant_in':'sum','quant_out':'sum', 'brand_inventory_in': 'max', 'brand_inventory_out': 'max'}
) # .sort_values(['unit_common_str'])

quant_in  \
main_1word main_2word_x       items_x                                            items_y                                            unit_common_str             
aci        berrak aci         berrak aci sus biberi tursusu 370 ml 1 x 12        berrak aci sus biberi tursusu 370 ml 1 x 12        adet eded ml         12.0   
acili      doritos acili      doritos acili 130 gr 1 x 16                        doritos acili 130 gr 1 x 16                        eded                220.0   
                              doritos acili 74 gr 1 x 24                         doritos acili 74 gr 1 x 24                         eded                220.0   
adjika     qlavprodukt adjika qlavprodukt adjika aromatnaya 170 qr 1 x 15        qlavprodukt adjika aromatnaya 170 qr 1 x 15        eded                840.0   
                              qlavprodukt adjika domasnaya 170 gr                qlavprodukt adjika aromatnaya 170 gr               ədəd eded           840.0   
...                                                                                                                                                       ...   
zolotoy    zolotoy petusok    zolotoy petusok naggets pendir 300 gr 1 x 12 4608  zolotoy petusok naggets pendir 300 gr 1 x 12 4608  eded                 71.0   
                              zolotoy petusok naggets toyuqlu 300 gr 1 x 12 4639 zolotoy petusok naggets toyuqlu 300 gr 1 x 12 4639 eded                 71.0   
                              zolotoy petusok naggets vkus.300 gr 1 x 12 4719    zolotoy petusok naggets vkus.300 gr 1 x 12 4719    eded                 71.0   
                              zolotoy petusok strips vkusnyaski 300 gr 1 x 12... zolotoy petusok strips vkusnyaski 300 gr 1 x 12... eded                 71.0   
                              zolotoy petusok suxari toyuq steyk 280 gr 1 x 1... zolotoy petusok suxari toyuq steyk 280 gr 1 x 1... eded                 71.0   

                                                                                                                                                     quant_out  \
main_1word main_2word_x       items_x                                            items_y                                            unit_common_str              
aci        berrak aci         berrak aci sus biberi tursusu 370 ml 1 x 12        berrak aci sus biberi tursusu 370 ml 1 x 12        adet eded ml           4.0   
acili      doritos acili      doritos acili 130 gr 1 x 16                        doritos acili 130 gr 1 x 16                        eded                  92.0   
                              doritos acili 74 gr 1 x 24                         doritos acili 74 gr 1 x 24                         eded                  92.0   
adjika     qlavprodukt adjika qlavprodukt adjika aromatnaya 170 qr 1 x 15        qlavprodukt adjika aromatnaya 170 qr 1 x 15        eded                 106.0   
                              qlavprodukt adjika domasnaya 170 gr                qlavprodukt adjika aromatnaya 170 gr               ədəd eded            106.0   
...                                                                                                                                                        ...   
zolotoy    zolotoy petusok    zolotoy petusok naggets pendir 300 gr 1 x 12 4608  zolotoy petusok naggets pendir 300 gr 1 x 12 4608  eded                  35.0   
                              zolotoy petusok naggets toyuqlu 300 gr 1 x 12 4639 zolotoy petusok naggets toyuqlu 300 gr 1 x 12 4639 eded                  35.0   
                              zolotoy petusok naggets vkus.300 gr 1 x 12 4719    zolotoy petusok naggets vkus.300 gr 1 x 12 4719    eded                  35.0   
                              zolotoy petusok strips vkusnyaski 300 gr 1 x 12... zolotoy petusok strips vkusnyaski 300 gr 1 x 12... eded                  35.0   
                              zolotoy petusok suxari toyuq steyk 280 gr 1 x 1... zolotoy petusok suxari toyuq s

In [55]:
compilation_scored[compilation_scored['items_y'] == 'sevimli dad rossiyskiy pendir 1 x 1']

,items_x,items_y,Quantity/Volume_x,unit_x,Quantity/Volume_y,unit_y,score1,score2,score3,score_w,...,Product or Service Name_x,Product or Service Name_y,unit_common,unit_common_set,unit_common_str,quant_in,quant_out,items_x_numberless,main_2word,main_1word
9926,sevimli dad rossiyskiy pendir 1 x 1,sevimli dad rossiyskiy pendir 1 x 1,16,adet.kg,"5,6",eded,100.0,100.0,100.0,1.0,...,SEVIMLI DAD ROSSIYSKIY PENDIR 1x1,SEVIMLI DAD ROSSIYSKIY PENDIR 1x1,adet.kg eded,"{adet, eded, kg}",adet eded kg,16.0,5.6,sevimli rossiyskiy pendir,sevimli rossiyskiy,sevimli


In [56]:
compilation_scored[compilation_scored['items_y'] == 'mia limon 1 x 12']

,items_x,items_y,Quantity/Volume_x,unit_x,Quantity/Volume_y,unit_y,score1,score2,score3,score_w,...,Product or Service Name_x,Product or Service Name_y,unit_common,unit_common_set,unit_common_str,quant_in,quant_out,items_x_numberless,main_2word,main_1word


In [57]:
compilation_scored[compilation_scored['items_y'] == '7 up 1 x 12']

,items_x,items_y,Quantity/Volume_x,unit_x,Quantity/Volume_y,unit_y,score1,score2,score3,score_w,...,Product or Service Name_x,Product or Service Name_y,unit_common,unit_common_set,unit_common_str,quant_in,quant_out,items_x_numberless,main_2word,main_1word


In [58]:
remove_numbers(simple_process_item('SEVIMLI.XAMA KAUNAS.20% 350GR AL 10% 200GR HDY 1X1'))

'sevimli xama kaunas'

In [59]:
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline
tokenizer = AutoTokenizer.from_pretrained("azizbarank/mbert-finetuned-azerbaijani-ner", is_split_into_words=True)
model = AutoModelForTokenClassification.from_pretrained("azizbarank/mbert-finetuned-azerbaijani-ner")
nlp = pipeline("token-classification", model=model, tokenizer=tokenizer, aggregation_strategy='simple')


In [60]:
for item in compilation_scored.sample(10)['items_x']:
    ner_results = nlp(item)
    print(item)
    [print(x['word'], ' ', x['entity_group']) for x in ner_results]
    print()

tadim misir 180 gr corn 1 x 12
tadim misir 180 gr corn 1 x 12   LABEL_0

nesquik snack latte 26 gr 1 x 20
nesquik   LABEL_5
snack   LABEL_6
latte 26 gr 1 x 20   LABEL_0

ulker puding alpella sokoladli 100 gr 1 x 24
ulker   LABEL_5
puding alpella sokoladli   LABEL_6
100 gr 1 x 24   LABEL_0

tadim klassik 75 qr govrulmus
tadim   LABEL_3
klassik 75 qr govrulmus   LABEL_4

president yagli 50 gr 1 x 60
president   LABEL_3
yagli   LABEL_4
50 gr 1 x 60   LABEL_0

ulker icim yogurt banan 45 gr
ulker   LABEL_3
icim yogurt banan 45 gr   LABEL_4

superfresh bamya sivri 450 gr 1 x 16
super   LABEL_1
##fre   LABEL_0
##sh   LABEL_1
bamya   LABEL_6
sivri 450 gr 1 x 16   LABEL_0

bebeto cobra x 30 gr 1 x 24
bebeto   LABEL_5
cobra   LABEL_6
x 30 gr 1 x 24   LABEL_0

sevimli dad kesmik yagli 9 pct 180 gr 1 x 1
sevimli   LABEL_3
dad kesmik yagli 9 pct 180 gr 1 x 1   LABEL_0

tadim fistixx taco yer fistigi 140 gr 1 x 12
tadim fistixx taco yer fistigi 140 gr 1 x 12   LABEL_0



In [61]:
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline
tokenizer = AutoTokenizer.from_pretrained("orgcatorg/bert-base-multilingual-cased-ner")
model = AutoModelForTokenClassification.from_pretrained("orgcatorg/bert-base-multilingual-cased-ner")
nlp = pipeline("ner", model=model, tokenizer=tokenizer)
example = 'token-classification'
ner_results = nlp(example)
ner_results

[{'entity': 'B-ORG',
  'score': 0.9993765,
  'index': 1,
  'word': 'tok',
  'start': 0,
  'end': 3},
 {'entity': 'B-ORG',
  'score': 0.99934775,
  'index': 2,
  'word': '##en',
  'start': 3,
  'end': 5},
 {'entity': 'I-ORG',
  'score': 0.9599418,
  'index': 3,
  'word': '-',
  'start': 5,
  'end': 6},
 {'entity': 'I-ORG',
  'score': 0.986715,
  'index': 4,
  'word': 'classification',
  'start': 6,
  'end': 20}]